<a href="https://colab.research.google.com/github/Arijit-datascience/pytorch_cifar10_resnet/blob/main/pytorch_cifar10_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Arijit-datascience/pytorch_cifar10_resnet
!pip install albumentations==1.0.0

Cloning into 'pytorch_cifar10_resnet'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 2), reused 14 (delta 2), pack-reused 0
Unpacking objects: 100% (17/17), done.
     |████████████████████████████████| 102kB 6.4MB/s 
     |████████████████████████████████| 38.2MB 76kB/s 
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


# New Section

In [5]:
from pytorch_cifar10_resnet import main as main
from pytorch_cifar10_resnet.model import resnet as models
from pytorch_cifar10_resnet.utils import utils as utils